# Module 5 - Getting to know your data

Hello, and welcome to this introductory course in Machine Learning for Biomedicine. To kick things off, I'd like to introduce you to [Weka](https://waikato.github.io/weka-wiki/downloading_weka/) one of the first Machine Learning platforms I used when I started out learning ML. Please take a minute and checkout [this great Weka video tutorial](https://waikato.github.io/weka-wiki/downloading_weka/)by Google. I think Weka is a great gateway platform that will undoubtedly take you a long way down these first steps into the ML ecosystem. In fact, Weka is still my goto platform for exploratory data analysis (EDA) which is the topic of today. Follow along the Weka example in the accompanying Lecture slides as we get to know the autism dataset distributed along with this notebook.

 ## What did we learn about the autism dataset using Weka
 1. We have 292 observations and 21 features
 2. The authors have developed a model using Demographics and 10 yes/no questions to Score patients for autism.
 3. Some features contain missing values.
 4. Appears the "relation" categorical variable was filled in by hand and has typos.
 5. Ethinicity for example is not uniformly sampled. Most patients are white.
 6. Most patients are 4yrs old or 11 with few patients in between.
 7. 2/3 of patients are Male.
 8. We can train a model with 100% accuracy using only the 10 yes/no questions.

**Questions**
+ How would you interpret the value of author's score metric? 
+ What is the precision and sensitivity of the author's model?
+ In our trained model using only the 10 yes/no questions, What score does a patient start off with before we start considering the answers to the 10 yes/no questions?
+ What score would a new patient get if they answers yes for all 10 question? 

# Formatting

Why was the data collected? Think bout how the data was intended to be used.
Who was the author? Can inform your understanding of the data’s purpose.
What does the literature say? Cohorts and datasets are often well studied. (important features, clinical need)
What do the values represent vs what are they trying to represent. 
Categorical VS continuous features – (African American, Caucasian, Hispanic ) vs (height, weight, age)

Distribution/frequency/density – 
	Boundaries
	Are there any holes
	What is distance between values/categories
	Infs and missing values
	Normally distributed
	Prevelance

Ordered:
Categorical on a scale of 1 to 5
Words in a sentence like the pangram “The quick brown fox jumps over the lazy dog.”
Amino acids in a protein sequence  
Time-ordered like the in the molecular dynamics at the Newtonian level or the exciton transport at the quantum level
Fluctuating stock prices 


QA/QC issues:
Was the data manually entered? – do features have values like “day1”, ”Day2”, day 3”
What are limits of detection for measurements? 1 millimole , 1 centimeter, 1 K, 1 second
Do measurements between days or sessions drift? Batch effects
Where are the feature outliers?
Do outliers in one feature tend to also be outliers in another?
Are the mean, median, and mode similar?
Are there any features with near zero variance?
Are there any overwhelming categories?


Correlations
How are features associated with the classes? Rsquared, t-tests
Do categories have similar class distributions?
Are  features correlated? Redundant data, Heatmaps, dimension reduction


Formatting, treat missing, transformations, feature selection -> Tidy data
